In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

import sys
import math
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary

import datetime
from sklearn import metrics
from sklearn.metrics import f1_score, precision_score, recall_score

In [2]:
from ce_generator import CE5min, CE15min, CE3min

In [3]:
# Check dataset sanity

n_train_data = 1000
ce5 = CE5min(n_train_data, 'train')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset()
print(ce_data.shape, ce_labels.shape, ae_labels.shape)
print(ce_labels[np.random.randint(n_train_data)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'

In [16]:
# Generate and save training data

# _, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset()
train_data = ae_labels
train_labels = ce_labels
# train_dataset = CEDataset(ae_labels, ce_labels)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)

In [17]:
idx=8231
train_data[idx], train_labels[idx]

count_dict={0:0, 1:0, 2:0, 3:0}
for temp in ce_labels:
    sum = np.sum(temp)
    count_dict[sum] += 1

count_dict

IndexError: index 8231 is out of bounds for axis 0 with size 1000

In [ ]:
np.save('./CE_dataset/nar_5min_train_data_{}.npy'.format(n_train_data), train_data)
np.save('./CE_dataset/nar_5min_train_labels_{}.npy'.format(n_train_data), train_labels)

In [ ]:
# Check dataset sanity

n_test_data = 1000
ce5 = CE5min(n_test_data, 'test')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset()
print(ce_data.shape, ce_labels.shape, ae_labels.shape)
print(ce_labels[np.random.randint(n_test_data)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'

In [ ]:
# Generate and save test data

_, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset()
test_data = ae_labels
test_labels = ce_labels

In [ ]:
np.save('./CE_dataset/nar_5min_test_data.npy', test_data)
np.save('./CE_dataset/nar_5min_test_labels.npy', test_labels)

In [ ]:
actions,action_labels

(['walk',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'sit',
  'flush_toilet',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'wash',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk',
  'walk'],
 [7,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  4,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  8,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7,
  7])

In [36]:
# Check dataset sanity

n_train_data = 10000
ce5 = CE5min(n_train_data, 'train')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(2)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

# print(len(action_data))
# print(len(event_labels))
# print(len(in_states))
# print(len(out_states))
# print(len(actions))
# print(len(action_labels))

print(windows)
print(t)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_1'], ['s3_2'], ['s3_2'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0']]
[['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_0'], ['s3_1'], ['s3_1'

In [37]:
# Generate and save training data

ce_data, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset(is_positive_sample=False)

id = np.random.randint(n_train_data)
print(ce_data[id], ce_labels[id], ae_labels[id])


# train_dataset = CEDataset(ae_labels, ce_labels)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)

count_dict={0:0, 1:0, 2:0, 3:0}
for temp in ce_labels:
    sum = np.max(temp)
    count_dict[sum] += 1

count_dict

[[0.         1.741951   0.         ... 0.34585917 0.5226191  0.49201033]
 [0.         1.7960644  0.         ... 0.27009666 0.58008677 0.35789058]
 [0.         1.8702769  0.         ... 0.40422195 0.46739814 0.4472375 ]
 ...
 [0.         0.5729566  0.         ... 0.10220088 0.1855685  0.25746548]
 [0.         0.56524754 0.         ... 0.10011166 0.18872088 0.254039  ]
 [0.         0.6390781  0.         ... 0.03346481 0.02251617 0.13684201]] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [5 5 5 5 5 5 5 5 7 7 7 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 8 0 0 8
 8 8 8 8 8 8 8 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]


{0: 4999, 1: 1667, 2: 1667, 3: 1667}

In [38]:
id = np.random.randint(n_train_data)
print(ce_data[id], ce_labels[id], ae_labels[id])

[[0.         0.8182661  0.         ... 0.26459593 0.4016924  0.32151362]
 [0.         1.675386   0.         ... 0.31873292 0.41584623 0.42078817]
 [0.         1.3948011  0.         ... 0.3717301  0.3141244  0.46758163]
 ...
 [0.         0.60391223 0.0292934  ... 0.13062444 0.22299616 0.27175066]
 [0.         0.22393486 0.0260104  ... 0.18588157 0.27706623 0.3284879 ]
 [0.         0.         0.04690205 ... 0.07008876 0.27741808 0.        ]] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [5 5 5 5 7 7 7 7 8 8 8 8 8 8 8 8 8 8 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 8 8 8 8 8 8 8 7 7 7 7 7 7 7]


In [39]:
train_data = ce_data
train_labels = ce_labels
np.save('./CE_dataset/ce5min_train_data_{}.npy'.format(n_train_data), train_data)
np.save('./CE_dataset/ce5min_train_labels_{}.npy'.format(n_train_data), train_labels)

In [ ]:
a = torch.rand(10,4)
a = torch.max(a, 1).values
b = []
b.append(a)
b.append(a)
b=np.concatenate(b)
b.shape

(20,)

In [40]:
# Check dataset sanity

n_test_data = 1000
ce5 = CE5min(n_test_data, 'test')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset()
print(ce_data.shape, ce_labels.shape, ae_labels.shape)
print(ce_labels[np.random.randint(n_test_data)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'

In [41]:
# Generate and save test data
ce_data, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset(is_positive_sample=False)
test_data = ce_data
test_labels = ce_labels
test_ae_labels = ae_labels

id = np.random.randint(n_test_data)
print(ce_data[id], ce_labels[id], ae_labels[id])


# train_dataset = CEDataset(ae_labels, ce_labels)
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False)

count_dict={0:0, 1:0, 2:0, 3:0}
for temp in ce_labels:
    sum = np.max(temp)
    count_dict[sum] += 1

count_dict

[[0.         0.51647943 0.         ... 0.51150584 0.3511231  0.24098411]
 [0.         0.5927707  0.         ... 0.4904068  0.35149896 0.24855715]
 [0.         1.1786895  0.01173378 ... 0.22257571 0.44003367 0.08466782]
 ...
 [0.         0.15709947 0.         ... 0.05866478 0.22725537 0.23003456]
 [0.         0.19891736 0.01816288 ... 0.24086955 0.2534365  0.3155847 ]
 [0.         0.         0.         ... 0.23939635 0.06111969 0.35643005]] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [5 5 5 5 5 5 5 7 8 8 8 8 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 8 8 0 0 8 8 8 8 8 8 8 7 7 7 7 7 7 7]


{0: 499, 1: 167, 2: 167, 3: 167}

In [42]:
np.save('./CE_dataset/ce5min_test_data.npy', test_data)
np.save('./CE_dataset/ce5min_test_labels.npy', test_labels)
# np.save('./CE_dataset/ce5min_test_ae_labels.npy', test_ae_labels)

In [ ]:
id = 0
test_ae_labels[id], test_labels[id]

(array([7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 5, 5,
        5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        4, 4, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [43]:
# Check dataset sanity

n_val_data = 2000
ce5 = CE5min(n_val_data, 'train')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

# ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset(is_positive_sample=True)
# print(ce_data.shape, ce_labels.shape, ae_labels.shape)
# print(ce_labels[np.random.randint(n_val_data)])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'

In [17]:
# Generate and save test data
ce_data, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset(is_positive_sample=False)

In [18]:
id = np.random.randint(n_val_data)
print(ce_data[id], ce_labels[id], ae_labels[id])
ce_data.shape

[[0.0000000e+00 8.4844917e-01 1.5893286e-01 ... 5.9983242e-01
  7.0030401e-03 3.2734177e-01]
 [0.0000000e+00 2.5877731e+00 0.0000000e+00 ... 7.2016090e-01
  4.6610677e-01 9.8205519e-01]
 [0.0000000e+00 2.4236705e+00 0.0000000e+00 ... 6.2114239e-01
  4.3387848e-01 9.3125206e-01]
 ...
 [0.0000000e+00 1.5507932e+00 4.4024922e-04 ... 1.6049965e-01
  5.1921916e-01 1.6357465e-01]
 [2.6111141e-02 1.7547989e+00 0.0000000e+00 ... 2.0729089e-01
  5.7212400e-01 3.4001520e-01]
 [0.0000000e+00 1.0003600e+00 0.0000000e+00 ... 2.1632096e-01
  4.4379482e-01 3.1787431e-01]] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [7 8 8 8 5 5 5 5 4 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 1 5 5 5 5 5 5 5 5 5 5 5 5 5 5]


(2000, 60, 128)

In [ ]:
np.save('./CE_dataset/nar_5min_val_data.npy'.format(n_val_data), ae_labels)
np.save('./CE_dataset/nar_5min_val_labels.npy'.format(n_val_data), ce_labels)

In [44]:
n_val_data = 1000
ce5 = CE5min(n_val_data, 'train')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'

In [45]:
# Generate and save test data
ce_data, ce_labels, ae_labels, _, _ = ce5.generate_CE_dataset(is_positive_sample=False)

In [46]:
id = np.random.randint(n_val_data)
print(ce_data[id], ce_labels[id], ae_labels[id])
ce_data.shape

[[0.         1.8147719  0.         ... 0.22704461 0.612273   0.37002188]
 [0.         0.9599924  0.         ... 0.31991875 0.2368501  0.44416073]
 [0.         0.5383582  0.         ... 0.         0.24352981 0.17169337]
 ...
 [0.         0.220536   0.02417441 ... 0.1922809  0.26670098 0.3346577 ]
 [0.         0.64465076 0.         ... 0.04061276 0.01536531 0.14184034]
 [0.         0.7093244  0.         ... 0.08428798 0.12736748 0.18481497]] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [5 5 5 5 5 5 5 5 7 7 7 8 8 8 8 0 0 0 8 8 0 0 8 8 8 8 8 8 8 8 8 8 8 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7]


(1000, 60, 128)

In [47]:
np.save('./CE_dataset/ce5min_val_data.npy'.format(n_val_data), ce_data)
np.save('./CE_dataset/ce5min_val_labels.npy'.format(n_val_data), ce_labels)

In [ ]:
n_train_data = 20000
ce5 = CE5min(n_train_data, 'train', simple_label=False)
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

# print(action_data)
print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'

In [ ]:
state_mapping = {'s1_0':0, 's1_1':1, 's2_0':2, 's2_1':3, 's2_2':4, 's3_0':5, 's3_1':6, 's3_2':7}
ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset(is_positive_sample=False, state_mapping=state_mapping)


In [ ]:
print(ce_data.shape, ce_labels.shape, in_states.shape, out_states.shape, ae_labels.shape)
id = np.random.randint(n_train_data)
print(ce_labels[id])
print(in_states[id])
print(out_states[id])

(20000, 60, 128) (20000, 60) (20000, 60) (20000, 60) (20000, 60)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 2 2 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[2 2 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]


In [ ]:
np.savez_compressed('./CE_dataset/state_nar_5min_full_train_{}.npz'.format(n_train_data), data=ae_labels, labels=ce_labels, in_states=in_states, out_states=out_states)

In [ ]:
n_test_data = 1000
ce5 = CE5min(n_test_data, 'test', simple_label=False)
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

# print(action_data)
print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

state_mapping = {'s1_0':0, 's1_1':1, 's2_0':2, 's2_1':3, 's2_2':4, 's3_0':5, 's3_1':6, 's3_2':7}
ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset(is_positive_sample=False, state_mapping=state_mapping)

print(ce_data.shape, ce_labels.shape, in_states.shape, out_states.shape, ae_labels.shape)
id = np.random.randint(n_test_data)
print(ce_labels[id])
print(in_states[id])
print(out_states[id])

np.savez_compressed('./CE_dataset/state_nar_5min_full_test.npz', data=ae_labels, labels=ce_labels, in_states=in_states, out_states=out_states)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'

In [ ]:
n_val_data = 1000
ce5 = CE5min(n_val_data, 'train', simple_label=False)
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

# print(action_data)
print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

state_mapping = {'s1_0':0, 's1_1':1, 's2_0':2, 's2_1':3, 's2_2':4, 's3_0':5, 's3_1':6, 's3_2':7}
ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset(is_positive_sample=False, state_mapping=state_mapping)

print(ce_data.shape, ce_labels.shape, in_states.shape, out_states.shape, ae_labels.shape)
id = np.random.randint(n_val_data)
print(ce_labels[id])
print(in_states[id])
print(out_states[id])

np.savez_compressed('./CE_dataset/state_nar_5min_full_val.npz', data=ae_labels, labels=ce_labels, in_states=in_states, out_states=out_states)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'

In [ ]:
n_train_data = 4000
ce5 = CE5min(n_train_data, 'train', simple_label=False)
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

# print(action_data)
print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

state_mapping = {'s1_0':0, 's1_1':1, 's2_0':2, 's2_1':3, 's2_2':4, 's3_0':5, 's3_1':6, 's3_2':7}
ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset(is_positive_sample=False, state_mapping=state_mapping)

print(ce_data.shape, ce_labels.shape, in_states.shape, out_states.shape, ae_labels.shape)
id = np.random.randint(n_train_data)
print(ce_data[id])
print(ce_labels[id])
print(in_states[id])
print(out_states[id])

np.savez_compressed('./CE_dataset/state_ce5min_full_train_{}.npz'.format(n_train_data), data=ce_data, labels=ce_labels, in_states=in_states, out_states=out_states)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'

In [ ]:
n_val_data = 1000
ce5 = CE5min(n_val_data, 'train', simple_label=False)
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

# print(action_data)
print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

state_mapping = {'s1_0':0, 's1_1':1, 's2_0':2, 's2_1':3, 's2_2':4, 's3_0':5, 's3_1':6, 's3_2':7}
ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset(is_positive_sample=False, state_mapping=state_mapping)

print(ce_data.shape, ce_labels.shape, in_states.shape, out_states.shape, ae_labels.shape)
id = np.random.randint(n_val_data)
print(ce_data[id])
print(ce_labels[id])
print(in_states[id])
print(out_states[id])

np.savez_compressed('./CE_dataset/state_ce5min_full_val.npz', data=ce_data, labels=ce_labels, in_states=in_states, out_states=out_states)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'

In [ ]:
n_test_data = 1000
ce5 = CE5min(n_test_data, 'test', simple_label=False)
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce5.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

# print(action_data)
print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

state_mapping = {'s1_0':0, 's1_1':1, 's2_0':2, 's2_1':3, 's2_2':4, 's3_0':5, 's3_1':6, 's3_2':7}
ce_data, ce_labels, ae_labels, in_states, out_states = ce5.generate_CE_dataset(is_positive_sample=False, state_mapping=state_mapping)

print(ce_data.shape, ce_labels.shape, in_states.shape, out_states.shape, ae_labels.shape)
id = np.random.randint(n_test_data)
print(ce_data[id])
print(ce_labels[id])
print(in_states[id])
print(out_states[id])

np.savez_compressed('./CE_dataset/state_ce5min_full_test.npz', data=ce_data, labels=ce_labels, in_states=in_states, out_states=out_states)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'

In [9]:
n_test_data = 1000
ce15 = CE15min(n_test_data, 'test', simple_label=False)
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce15.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

# print(action_data)
print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

state_mapping = {'s1_0':0, 's1_1':1, 's2_0':2, 's2_1':3, 's2_2':4, 's3_0':5, 's3_1':6, 's3_2':7}
ce_data, ce_labels, ae_labels, in_states, out_states = ce15.generate_CE_dataset(is_positive_sample=False)

print(ce_data.shape, ce_labels.shape, in_states.shape, out_states.shape, ae_labels.shape)
id = np.random.randint(n_test_data)
print(ce_data[id])
print(ce_labels[id])
print(in_states[id])
print(out_states[id])


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']

In [ ]:
count_dict={0:0, 1:0, 2:0, 3:0}
for temp in ce_labels:
    sum = np.sum(temp)
    count_dict[sum] += 1

count_dict

{0: 659, 1: 102, 2: 122, 3: 117}

In [ ]:
np.savez_compressed('./CE_dataset/state_ce15min_full_test.npz', data=ce_data, labels=ce_labels, in_states=in_states, out_states=out_states)
# np.savez_compressed('./CE_dataset/state_ce15min_test.npz', data=ce_data, labels=ce_labels, in_states=in_states, out_states=out_states)

In [10]:
n_test_data = 1000
ce15 = CE15min(n_test_data, 'test')
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce15.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']

In [11]:
# Generate and save test data
ce_data, ce_labels, ae_labels, _, _ = ce15.generate_CE_dataset(is_positive_sample=False)

id = np.random.randint(n_test_data)
print(ce_data[id], ce_labels[id], ae_labels[id])

count_dict={0:0, 1:0, 2:0, 3:0}
for temp in ce_labels:
    sum = np.max(temp)
    count_dict[sum] += 1

count_dict

[[0.         0.77674526 0.         ... 0.41426092 0.3965854  0.17228368]
 [0.         0.82263505 0.         ... 0.37624466 0.38896936 0.1855558 ]
 [0.         0.77674526 0.         ... 0.41426092 0.3965854  0.17228368]
 ...
 [0.         0.78924865 0.         ... 0.2723183  0.11406017 0.37495944]
 [0.         0.18859468 0.         ... 0.11252563 0.2699081  0.15629223]
 [0.         0.15741365 0.         ... 0.16984029 0.25779676 0.16697519]] [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0] [5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 7 8 8

{0: 499, 1: 167, 2: 167, 3: 167}

In [12]:
np.save('./CE_dataset/ce15min_test_data.npy', ce_data)
np.save('./CE_dataset/ce15min_test_labels.npy', ce_labels)

In [13]:
n_test_data = 1000
ce3 = CE3min(n_test_data, 'test', simple_label=False)
action_data, event_labels, in_states, out_states, actions, action_labels, windows, t = ce3.generate_event(0)
# for a,l in zip(actions, labels):
#     print(a,l)

# print(action_data)
print(event_labels)
print(in_states)
print(out_states)
print(actions)
print(action_labels)

print(len(action_data))
print(len(event_labels))
print(len(in_states))
print(len(out_states))
print(len(actions))
print(len(action_labels))

print(windows)
print(t)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
[['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_1'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0'], ['s1_0']]
['walk', 'walk', 'wash', 'sit', 'sit', 'flush_toilet', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'walk', 'sit', 'sit', 'walk', 'walk', 'walk', 'type', 'type', 'typ

In [15]:
ce_data, ce_labels, ae_labels, in_states, out_states = ce3.generate_CE_dataset(is_positive_sample=False)

print(ce_data.shape, ce_labels.shape, in_states.shape, out_states.shape, ae_labels.shape)
id = np.random.randint(n_test_data)
print(ce_data[id])
print(ce_labels[id])
# print(in_states[id])
# print(out_states[id])

count_dict={0:0, 1:0, 2:0, 3:0}
for temp in ce_labels:
    sum = np.max(temp)
    count_dict[sum] += 1

count_dict


(1000, 36, 128) (1000, 36) (1000, 36, 1) (1000, 36, 1) (1000, 36)
[[0.         1.10881    0.         ... 0.40923506 0.5296134  0.1853222 ]
 [0.         0.15741365 0.         ... 0.16984029 0.25779676 0.16697519]
 [0.         0.5607676  0.         ... 0.08968978 0.02746379 0.2087098 ]
 ...
 [0.         0.55273163 0.         ... 0.07437789 0.02603179 0.20626377]
 [0.         0.5607676  0.         ... 0.08968978 0.02746379 0.2087098 ]
 [0.         0.5407525  0.         ... 0.07416808 0.0232014  0.20614325]]
[0 0 0 0 0 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


{0: 499, 1: 167, 2: 167, 3: 167}

In [16]:
np.save('./CE_dataset/ce3min_test_data.npy', ce_data)
np.save('./CE_dataset/ce3min_test_labels.npy', ce_labels)